In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
test_csv = pd.read_csv("ruw_data/test.csv")
submission_csv = pd.read_csv("prediction/submission.csv")

In [13]:
# test_csv.head()
# submission_csv.head()

,id,sentence,subject_entity,object_entity,label
0,317,자포리자주 주지사 올렉산드르 스타루흐는 이 도시에서 지역 방위군과의 총격전이 여전히...,"{'word':'올렉산드르 스타루흐','start_idx':10,'end_idx':...","{'word':'주지사','start_idx':6,'end_idx':8,'type'...",per:title
1,21,"2012년, 푸틴은 3선 대통령을 노렸고, 거의 64%에 달하는 득표율로 대선에서 ...","{'word':'푸틴','start_idx':7,'end_idx':8,'type':...","{'word':'3선 대통령','start_idx':11,'end_idx':16,'...",per:title
2,342,북아시아 전체와 동유럽의 대부분에 11개의 시간대에 걸친 영토가 있다.,"{'word':'북아시아','start_idx':0,'end_idx':3,'type...","{'word':'동유럽','start_idx':9,'end_idx':11,'type...",no_relation
3,265,우크라이나에 군사 개입과 크림 반도의 병합.,"{'word':'우크라이나','start_idx':0,'end_idx':4,'typ...","{'word':'크림 반도','start_idx':14,'end_idx':18,'t...",no_relation
4,557,"하지만, 1991년까진 공산당 외 다른 정당은 금지되어 있었다.","{'word':'공산당','start_idx':13,'end_idx':15,'typ...","{'word':'1991년','start_idx':5,'end_idx':9,'typ...",no_relation


## 1. 정답과 모델의 예측값 비교하기

In [3]:
sentences = test_csv["sentence"]
subject_entities, object_entities = test_csv["subject_entity"], test_csv["object_entity"] 

my_labels = submission_csv["pred_label"]
ans_labels = test_csv["label"]

In [4]:
i = -1
cnt = 0

while i < len(my_labels)-1:
    i += 1
    if my_labels[i] == ans_labels[i]:
        continue
    cnt += 1
    print(f"{cnt}: {sentences[i]}")
    print(f"sub:{eval(subject_entities[i])['word']}, obj:{eval(object_entities[i])['word']}")
    print(f"예측: {my_labels[i]}, sota: {ans_labels[i]}")
    print()
print(f"num of diff: {cnt} / {i}")

1: 이에 형식상으로는 소련에서 가장 큰 공화국이 탈퇴한 것으로 보였으나 러시아 측에서는 '더 이상 존재하지 않는 나라'에서 탈퇴하는 것은 불가능하므로 이 같은 말은 사실이 아니라고 말했다.
sub:러시아, obj:소련
예측: no_relation, sota: org:member_of

2: 같은 날 지역 가스 공급 업체의 발표에 따르면, 루한스크 주의 약 10,000 가구 이상이 가스관 손상으로 가스를 받지 못한다고 말했다.
sub:가스 공급 업체, obj:가스
예측: org:property, sota: no_relation

3: 이 결정으로 12월 12일부터 25일까지 열릴 의회 정기국회에 헌법 개정안을 비준하기로 하였다.
sub:의회 정기국회, obj:12월 12일
예측: eve:date, sota: no_relation

4: 하자르 카간국의 영향에 따라 동슬라브족으로서의 정체성이 생기기 시작하였고 8세기에서 9세기에 루스 카간국이라는 고대 투르크어인 군주 칭호인 카간을 자칭하는 북게르만족 루스인의 첫 국가가 등장하였다.
sub:동슬라브족, obj:북게르만족
예측: org:member_of, sota: no_relation

5: 1991년 1월 14일, 니콜라이 리즈코프가 '소련의 총리'직으로 불리는 소련 각료평의회 주석에서 사임하였고, 그를 이어 발렌틴 파블로프가 소련 총리직에 취임하였다.
sub:니콜라이 리즈코프, obj:발렌틴 파블로프
예측: per:title, sota: no_relation

6: 우크라이나 내무부는 우크라이나군이 4월 23일 슬로비얀스크 근처의 시비아토고르스크(Sviatogorsk) 도시를 탈환했다고 말했다.
sub:우크라이나군, obj:우크라이나 내무부
예측: org:property, sota: no_relation

7: 이 시위는 러시아의 크림 공화국 합병으로 인해 우크라이나의 남동부 지역의 대규모 친러시아 시위로 발전했으며, 이후 도네츠크 인민공화국과 루간스크 인민공화국이 분리주의를 제창하며 독립함에 따라 우크라이나

## 2. f1 score 계산

In [6]:
from sklearn.metrics import precision_score

micro_f1 = precision_score(ans_labels, my_labels, average='micro')
macro_f1 = precision_score(ans_labels, my_labels, average='macro')
weighted_f1 = precision_score(ans_labels, my_labels, average='weighted')
average_f1 = precision_score(ans_labels, my_labels, average=None)

print('micro f1 score:', micro_f1)
print('macro f1 score:', macro_f1)
print('weighted f1 score:', weighted_f1)
print('f1 score of each class:', average_f1)

micro f1 score: 0.847457627118644
macro f1 score: 0.7782325797772022
weighted f1 score: 0.8441267606027331
f1 score of each class: [0.92105263 1.         0.84782609 1.         0.         0.75
 0.74358974 0.76923077 0.78571429 0.96491228]
